<a href="https://colab.research.google.com/github/spiegel-jpg/Denial_of_Service-8240/blob/main/CSCI8240RP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import choice
from scipy.stats import norm
import statistics
from google.colab import drive
import pandas as pd
import plotly.express as px
import math
import pywt

In [ ]:
drive.mount('/content/drive') # Mounting the drive

In [ ]:
proj_dir = '/content/drive/MyDrive/spiegel'

In [ ]:
data = pd.read_excel(proj_dir+'/get.xlsx') # Reading the data

In [ ]:
data['Time'] = data['Time'].dt.floor('h')
timeseries = data.set_index('Time')

In [ ]:
H = 10634                          # Total hours of data we have
pages = pd.read_csv(proj_dir+'/pages.csv')
valid_page = pages['PATH']

In [ ]:
pages = pd.read_csv(proj_dir+'/pages.csv')
valid_page = pages['PATH']

# Creating quantiles with 114 pages each.

quantile1 = list(valid_page[0:114])
quantile2 = list(valid_page[114:228])
quantile3 = list(valid_page[228:342])
quantile4 = list(valid_page[342:456])
quantile5 = list(valid_page[456:570])

In [ ]:
valid_page_checker = set(valid_page)
N = len(pages)                      # N is the total pages on the website
                                    # These are all the pages which are services with 200 HTML status code
M = int(5)
no_of_quantiles = int(N/M)

#  fij represent the total number of requests on ith page (1 ≤ i ≤ N) in the jth hour.


In [ ]:
# U is the quantiles 
# timeseries.index[i]
time=set()
for i in range(len(timeseries)):
  time.add(timeseries.index[i])
time = list(sorted(time))
# print(time)
# U = [[0 for i in range(M)] for j in range(int(no_of_quantiles))]
# for i in range(int(no_of_quantiles)):
#   for j in range(M):
#     U[i][j]


In [ ]:
S = [[0 for i in range(int(H))] for j in range(int(M))] 

In [ ]:
for i in range(H):
  requests = timeseries.loc[time[i]]
  no_of_req = int(len(requests))
  request_path = requests['Path']
  request_response = requests['Response']
  if not isinstance(requests, pd.DataFrame):
    if request_response == 200:
        if request_path in quantile1:
          S[0][i]+=1
        elif request_path in quantile2:
          S[1][i]+=1
        elif request_path in quantile3:
          S[2][i]+=1
        elif request_path in quantile4:
          S[3][i]+=1
        elif request_path in quantile5:
          S[4][i]+=1
  else:
    for j in range(no_of_req):
      if request_response[j] == 200:
        if request_path[j] in quantile1:
          S[0][i]+=1
        elif request_path[j] in quantile2:
          S[1][i]+=1
        elif request_path[j] in quantile3:
          S[2][i]+=1
        elif request_path[j] in quantile4:
          S[3][i]+=1
        elif request_path[j] in quantile5:
          S[4][i]+=1

In [ ]:
x = np.arange(0, H, 1)
plt.plot(x,S[0])
plt.title("S1")
plt.xlabel("Time(n)")
plt.ylabel("No of requests")

In [ ]:
x = np.arange(0, H, 1)
plt.plot(x,S[1])
plt.title("S2")
plt.xlabel("Time(n)")
plt.ylabel("No of requests")

In [ ]:
x = np.arange(0, H, 1)
plt.plot(x,S[2])
plt.title("S3")
plt.xlabel("Time(n)")
plt.ylabel("No of requests")

In [ ]:
x = np.arange(0, H, 1)
plt.plot(x,S[3])
plt.title("S4")
plt.xlabel("Time(n)")
plt.ylabel("No of requests")

In [ ]:
x = np.arange(0, H, 1)
plt.plot(x,S[4])
plt.title("S5")
plt.xlabel("Time(n)")
plt.ylabel("No of requests")

For Noise


Rtotal is the total number of requests for H/2 h. Hence, Rtotal is 129616 requests.

In [ ]:
R_total_no = int(H/2)
noise1 = S[0][0:R_total_no]
noise2 = S[1][0:R_total_no]
noise3 = S[2][0:R_total_no]
noise4 = S[3][0:R_total_no]
noise5 = S[4][0:R_total_no]
noise = (noise1+noise2+noise3+noise4+noise5)

In [ ]:
# Getting the Mean 
noise1_mean = np.mean(noise1)
noise2_mean = np.mean(noise2)
noise3_mean = np.mean(noise3)
noise4_mean = np.mean(noise4)
noise5_mean = np.mean(noise5)
# Getting the Variance
noise1_var = np.var(noise1)
noise2_var = np.var(noise2)
noise3_var = np.var(noise3)
noise4_var = np.var(noise4)
noise5_var = np.var(noise5)
# Getting mean and variance for all quantiles
noise_mean = np.mean(noise)
noise_var = np.var(noise)
print(noise2_var)
print(noise_mean, noise_var)

In [ ]:
# Normal distribution for one of the quantiles according to the formula in the paper.
x = np.arange(-R_total_no, R_total_no, 1)
p=0.2 # 0.03 0.05 0.1 0.15 0.2
plt.plot(x,norm.pdf(x,p*noise_mean,p*noise_var))
# we havent used normal distribution in our paper as the paper says to add random noise but the normal distribution is will add a bell shaped curve
# instead we took the approach of adding noise at random intervals with requests being added at a probability corresponding to the mean of the request for that quantile 

In [ ]:
x = np.arange(0, H, 1)
plt.plot(x,S[0])

In [ ]:
# A_noise1 = np.random.normal(0.03*noise_mean, 0.03*noise_var,4254)/M
# A_noise2 = np.random.normal(0.05*noise_mean, 0.05*noise_var,4254)/M
# A_noise3 = np.random.normal(0.1*noise_mean, 0.1*noise_var,4254)/M
# A_noise4 = np.random.normal(0.15*noise_mean, 0.15*noise_var,4254)/M
# A_noise5 = np.random.normal(0.2*noise_mean, 0.2*noise_var,4254)/M

In [ ]:
# S0_40noise = S[0][5316:9570]+A_noise3
# S0_paper_noise=[]
# for i in range(H):
#   if i<5317 or i>9569:
#     S0_paper_noise.append(S[0][i])
#   elif i>5316 and i<9570:
#     S0_paper_noise.append(S0_40noise[i-5317])
# df = pd.DataFrame(dict(
#     x = np.arange(0,H,1),
#     y = np.array(S0_paper_noise)
# ))
# fig = px.line(df, x="x", y="y", title="Quantile 1 With added noise according to papers FRC attack") 
# fig.show(0)
# x = np.arange(0,H,1)
# y = np.array(S0_paper_noise)-np.array(S[0])
# plt.title('Quantile 1 noise 10% attack')
# plt.plot(x,y)

### **Here starts the real thing (Graph)**




In [ ]:
# Quantile 1 with added noise

quantile1_req_noise=[]
quantile1_prob_noise=[]
P=0.01
for i in range(math.floor(noise1_var*P)):
  quantile1_req_noise.append(i+1)
  quantile1_prob_noise.append(1/(noise1_var*P))
quantile1_prob_noise.pop()
for_zero = 1 - sum(quantile1_prob_noise)
quantile1_prob_noise.insert(0,for_zero)
noise1_data = choice(quantile1_req_noise, 4254, p=quantile1_prob_noise) # noise
S0_mod = noise1_data + S[0][5317:9571]
S0=[]  # sequence 0 with added noise
for i in range(H):
  if i<5317:
    S0.append(S[0][i])
  elif i>5316 and i<9570:
    S0.append(S0_mod[i-5317])
  elif i>9569:
    S0.append(S[0][i])
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S0
))
df1 = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = np.array(S0) - np.array(S[0])
))

# new_df = df - df_no_noise
fig = px.line(df, x="x", y="y", title="Quantile 1 With added noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)
fig1 = px.line(df1, x="x", y="y", title="Quantile 1 noise") 
fig1.add_vrect(x0=5316, x1=9570)
fig1.show(0)
# plt.plot(np.arange(0,H,1), S0)


In [ ]:
# Quantile 1 without noise
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S[0]
))
fig = px.line(df, x="x", y="y", title="Quantile 1 Without noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
# Quantile 2 with added noise
quantile2_req_noise = []
quantile2_prob_noise = []
P=0.05
for i in range(math.floor(noise2_var)):
  quantile2_req_noise.append(i)
  quantile2_prob_noise.append(P/(noise2_var))
quantile2_prob_noise.pop()
for_zero = 1 - sum(quantile2_prob_noise)
quantile2_prob_noise.insert(0,for_zero)

noise2_data = choice(quantile2_req_noise, 4254, p=quantile2_prob_noise) # noise
S1_mod = noise2_data + S[1][5317:9571]
S1=[]
for i in range(H):
  if i<5317:
    S1.append(S[1][i])
  elif i>5316 and i<9570:
    S1.append(S1_mod[i-5317])
  elif i>9569:
    S1.append(S[1][i])
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S1
))
df1 = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = np.array(S1) - np.array(S[1])
))
fig = px.line(df, x="x", y="y", title="Quantile 2 With added noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)
fig1 = px.line(df1, x="x", y="y", title="Quantile 2 noise") 
fig1.add_vrect(x0=5316, x1=9570)
fig1.show(0)

In [ ]:
# Quantile 2 without noise
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S[1]
))
fig = px.line(df, x="x", y="y", title="Quantile 2 Without noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
# Quantile 3 with added noise
quantile3_req_noise = [0,1]
noise3_data = choice(quantile3_req_noise, 4254, p=[1-noise3_mean,noise3_mean]) # noise
S2_mod = noise3_data + S[2][5317:9571]
S2=[]
for i in range(H):
  if i<5317:
    S2.append(S[2][i])
  elif i>5316 and i<9570:
    S2.append(S2_mod[i-5317])
  elif i>9569:
    S2.append(S[2][i])
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S2
))
fig = px.line(df, x="x", y="y", title="Quantile 3 With added noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
# Quantile 3 without noise
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S[2]
))
fig = px.line(df, x="x", y="y", title="Quantile 3 Without noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
# Quantile 4 with added noise
quantile4_req_noise = [0,1]
noise4_data = choice(quantile4_req_noise, 4254, p=[1-noise4_mean,noise4_mean]) # noise
S3_mod = noise4_data + S[3][5317:9571]
S3=[]
for i in range(H):
  if i<5317:
    S3.append(S[3][i])
  elif i>5316 and i<9570:
    S3.append(S3_mod[i-5317])
  elif i>9569:
    S3.append(S[3][i])
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S3
))
fig = px.line(df, x="x", y="y", title="Quantile 4 With added noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
# Quantile 4 without noise
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S[3]
))
fig = px.line(df, x="x", y="y", title="Quantile 4 without noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
quantile5_req_noise = [0,1]
noise5_data = choice(quantile5_req_noise, 4254, p=[1-noise5_mean,noise5_mean]) # noise
# plt.plot(np.arange(5317,9571,1),S[0][5317:9571]+data)
S4_mod = noise5_data + S[4][5317:9571]
S4=[]
for i in range(H):
  if i<5317:
    S4.append(S[4][i])
  elif i>5316 and i<9570:
    S4.append(S4_mod[i-5317])
  elif i>9569:
    S4.append(S[4][i])
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S4
))
fig = px.line(df, x="x", y="y", title="Quantile 5 With added noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S[4]
))
fig = px.line(df, x="x", y="y", title="Quantile 5 without noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
print("Darpan The Dirbuster")

In [ ]:
target_seq = []
for i in range(H):
  if i<5317:
    target_seq.append(0)
  elif i>5316 and i<9570:
    target_seq.append(1)
  elif i>9569:
    target_seq.append(0)
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = target_seq
))
fig = px.line(df, x="x", y="y", title="Target sequence") 
# fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

**Thus we have added noise to the data with a probability distribution depending on the data from the first 50% data range. We have currently not accounted the different attack percentages as it is still unclear with our own implementation.**

In [ ]:
print(pywt.families(short=False))
wavelet = pywt.Wavelet('db4')
print(wavelet)

In [ ]:
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S[0]
))
fig = px.line(df, x="x", y="y", title="Quantile 1 Without noise") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
a,d = pywt.dwt(S0,'db4')
# d=[0]*5320;
d.fill(0)
S0r = pywt.idwt(a,d,'db4')
for i in range(100):
  a,d = pywt.dwt(S0r,'db4')
  d=[0]*5320;
  S0r = pywt.idwt(a,d,'db4')


for i in range(len(S0r)):
  if S0r[i]<=0:
    S0r[i]=0
Dk1=S0r
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S0r
))
fig = px.line(df, x="x", y="y", title="Denoise S0") 
fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
a,d = pywt.dwt(S1,'db4')
d.fill(0)
S0r = pywt.idwt(a,d,'db4')
for i in range(100):
  a,d = pywt.dwt(S0r,'db4')
  d=[0]*5320;
  S0r = pywt.idwt(a,d,'db4')


for i in range(len(S0r)):
  if S0r[i]<=0:
    S0r[i]=0
Dk2=S0r
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S0r
))
fig = px.line(df, x="x", y="y", title="Denoise S1") 
# fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
a,d = pywt.dwt(S2,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')

for i in range(len(S0r)):
  if S0r[i]<=0:
    S0r[i]=0
Dk3=S0r
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S0r
))
fig = px.line(df, x="x", y="y", title="Denoise S2") 
# fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
a,d = pywt.dwt(S3,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')

for i in range(len(S0r)):
  if S0r[i]<=0:
    S0r[i]=0
Dk4=S0r
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S0r
))
fig = px.line(df, x="x", y="y", title="Denoise S3") 
# fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

In [ ]:
a,d = pywt.dwt(S4,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')
a,d = pywt.dwt(S0r,'db4')
d=[0]*5320;
S0r = pywt.idwt(a,d,'db4')

for i in range(len(S0r)):
  if S0r[i]<=0:
    S0r[i]=0
Dk5=S0r
df = pd.DataFrame(dict(
    x = np.arange(0,H,1),
    y = S0r
))
fig = px.line(df, x="x", y="y", title="Denoise S4") 
# fig.add_vrect(x0=5316, x1=9570)
fig.show(0)

LSTM Implementation

In [ ]:
# Dk1 Dk2 Dk3 Dk4 Dk5 are the final sequences to be fed into the network 
# all combined make the signal and should give us 1 model
# target